In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
#for start and end
SOS_token=0
EOS_token=1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2
    
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1  
             


In [3]:
#The reason for the unicodeToascii function is to convert Unicode strings to plain ASCII by removing special characters, accents, and diacritical marks (like é, ü, ñ, etc.).
def unicodeToascii(s):
    return ''.join(
        c for c in  unicodedata.normalize('NFD',s)
        if unicodedata.category(c)!='Mn'
    )
def normalizeString(s):
    s=unicodeToascii(s.lower().strip())
    s=re.sub(r"([.!?])",r" \1",s) #Adds a space before each punctuation mark.
    s=re.sub(r"[^a-zA-z!?]+",r" ", s)#Removes any character that is not:A-Z a-z . ! ? replace with " "
    return s.strip()

In [4]:
def readlines(lang1, lang2, reverse=False):
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n') #split the text into the lines

    pairs=[[normalizeString(s) for s in l.split('\t')] for l in lines]   #normalize and clean the sentences for traning 

    if reverse:
        pairs=[list(reversed(p)) for p in pairs]
        input_lang=Lang(lang2)
        output_lang=Lang(lang1)
    else:
        input_lang=Lang(lang1)
        output_lang=Lang(lang2)   
    return input_lang,output_lang,pairs    

In [5]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)
#Complex sentences or rare structures may introduce noise, making it harder for the model to generalize effectively.
#Lowers the computational cost of training a model.By focusing on the simple 
#and  well-aligned sentences ensures the model learns clean and accurate mappings.
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [17]:
def prepareData(lang1,lang2 ,reverse=False):
    input_lang,output_lang, pairs=readlines(lang1,lang2, reverse)
    pairs = filterPairs(pairs)
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)    
    return input_lang, output_lang, pairs

In [20]:
class Encoder(nn.Module):
    def __init__(self,input_size,hidden_size,dropout_p=0.1):
        super(Encoder,self).__init__()
        self.hidden_size=hidden_size

        self.embedding=nn.Embedding(input_size,hidden_size)
        self.gru =nn.GRU(hidden_size,hidden_size,batch_first=True)
        self.dropout=nn.Dropout(dropout_p)
    def forward(self,input): 
        embedded=self.dropout(self.embedding(input))
        output,hidden=self.gru(embedded)
        return output,hidden   

In [8]:
class Decoder(nn.Module):
    def __init__(self,hidden_size,output_size):
        super(Decoder,self).__init__()
        self.embedding= nn.Embedding(output_size, hidden_size)
        self.gru=nn.GRU(hidden_size,hidden_size,batch_first=True)
        self.out=nn.Linear(hidden_size,output_size)
    def forward(self,endcoder_outputs,encoder_hidden,target_tensor=None):
        batch_size=endcoder_outputs.size(0)
        #Starting input is initialize as zeros
        decoder_input=torch.empty(batch_size,1,dtype=torch.long,device=device).fill_(SOS_token)
       #encoder output initialize the decoder starting hidden state  
        decoder_hidden=encoder_hidden
        decoder_outputs=[]

        for i in range(MAX_LENGTH):
            decoder_output,decoder_hidden=self.forward_step(decoder_input,decoder_hidden)
           #output contains the the logit for the every timestep of vocab size and which ever having large probability that word's embedding is send to the next timestep 
            decoder_outputs.append(decoder_output)
            if(target_tensor is not None): #teacher forcing
                decoder_input=target_tensor[:,i].unsqueeze(1)
            else:

                _,topi=decoder_output.topk(1)#select the most preferable token for the next step input
          
                decoder_input=topi.sequeeze(-1).detach()
        #Combines the logits from all time steps into a single tensor of shape (batch_size, MAX_LENGTH, vocab_size).
        decoder_outputs=torch.cat(decoder_outputs,dim=1)
        #Converts logits into log-probabilities for the vocabulary. This is useful for training with negative log-likelihood loss.
        decoder_outputs=F.log_softmax(decoder_outputs,dim=-1)
        return decoder_outputs,decoder_hidden,None     
    def forward_step(self,input,hidden):
        output =self.embedding(input)
        output=F.relu(output)
        output,hidden=self.gru(output,hidden)
        output=self.out(output)
        return output,hidden          

In [23]:
# #Imagine a stage performance where multiple actors (input tokens) are acting at once. The decoder (query) represents the director who decides which actor to focus on.

# Query: The director's current instructions (what's needed right now).
# Keys: The actors' performances (encoder outputs).
# Neural Network (Bahdanau Attention): The spotlight operator who decides, based on the director's instructions, which actor should be
# highlighted. The weights determine how much focus each actor gets.
class BahdanauAttention(nn.Module):
  #Instead, it transforms them separately (using Wa and Ua) and then combines them (via addition) before applying tanh
  #Let the network learn distinct transformations for the query and keys before combining them.  
  #Bahdanau’s method focuses on the element-wise interaction (via addition) of the query and keys,
  #as this preserves more individual information about each. Concatenation tends to mix the representations more, which may lose some structure but can still work depending on the task.


    def __init__(self,hidden_size):
        super(BahdanauAttention,self).__init__()
        self.Wa =nn.Linear(hidden_size,hidden_size)
        self.Ua=nn.Linear(hidden_size,hidden_size)
        self.Va=nn.Linear(hidden_size,1)
    def forward(self,query ,keys):   
        scores=self.Va(torch.tanh(self.Wa(query)+self.Ua(keys)))
        scores =scores.squeeze(2).unsqueeze(1)#squeeze :remove the dimension at specific pos of dim=1,if not specified it will Removes dimensions of size 1 from a tensor


        weights=F.softmax(scores,dim=-1)#across the squence length
        context =torch.bmm(weights,keys)#(batch matrix multiplication)The result is a single context vector for each query, summarizing the encoder information weighted by the attention scores.
        return context,weights 
    

class AttnDecoder(nn.Module):
    def __init__(self,hidden_size,output_size,dropout_p=0.1):
        super(AttnDecoder,self).__init__()
        self.embedding=nn.Embedding(output_size,hidden_size)
        self.attention =BahdanauAttention(hidden_size)
        self.gru =nn.GRU(2*hidden_size,hidden_size,batch_first=True)
        self.out=nn.Linear(hidden_size,output_size)
        self.dropout =nn.Dropout(dropout_p)
    def forward(self,encoder_outputs,encoder_hidden,target_tensor=None):
         batch_size=encoder_outputs.size(0)
         decoder_input=torch.empty(batch_size,1,dtype=torch.long,device=device).fill_(SOS_token)
         decoder_hidden=encoder_hidden 
         decoder_outputs=[]
         attentions=[]

         for i in range(MAX_LENGTH):
             decoder_output,decoder_hidden,attn_weights=self.forward_step(decoder_input,decoder_hidden,encoder_outputs)
             decoder_outputs.append(decoder_output)
             attentions.append(attn_weights)


             if target_tensor is not None:
                 decoder_input=target_tensor[:,i].unsqueeze(1)
             else:
                 _,topi=decoder_output.topk(1)
                 decoder_input=topi.squeeze(-1).detach()



         decoder_outputs = torch.cat(decoder_outputs, dim=1)
         decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
         attentions = torch.cat(attentions, dim=1)

         return decoder_outputs, decoder_hidden, attentions    

    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights


In [10]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)


def tensorsFromPair(pair, input_lang, output_lang):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader



In [11]:
import matplotlib.pyplot as plt
import numpy as np

def showPlot(points):
    # Create the figure and axes
    fig, ax = plt.subplots()

    # Plot the points
    ax.plot(points)

    # Optional: Customize labels and title
    ax.set_xlabel("Epochs")
    ax.set_ylabel("Loss")
    ax.set_title("Training Loss over Epochs")

    # Show the plot
    plt.show()


In [12]:
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn

def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001, plot_every=100):
    plot_losses = []
    plot_loss_total = 0  # Reset every plot_every

    # Define optimizers and loss function
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    # Training loop
    for epoch in range(1, n_epochs + 1):
        # Initialize tqdm progress bar for the epoch
        epoch_loss = 0
        with tqdm(train_dataloader, desc=f"Epoch {epoch}/{n_epochs}", unit="batch") as progress_bar:
            for data in progress_bar:
                input_tensor, target_tensor = data
                encoder_optimizer.zero_grad()
                decoder_optimizer.zero_grad()

                # Forward pass
                encoder_outputs, encoder_hidden = encoder(input_tensor)
                decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

                # Calculate loss
                loss = criterion(
                    decoder_outputs.view(-1, decoder_outputs.size(-1)),
                    target_tensor.view(-1)
                )
                loss.backward()

                # Update weights
                encoder_optimizer.step()
                decoder_optimizer.step()

                # Accumulate batch loss
                batch_loss = loss.item()
                epoch_loss += batch_loss

                # Update progress bar with the current batch loss
                progress_bar.set_postfix(loss=batch_loss)

        # Calculate average loss for the epoch
        avg_loss = epoch_loss / len(train_dataloader)
        plot_loss_total += avg_loss

        # Update plot losses periodically
        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    # Display the loss plot at the end
    showPlot(plot_losses)


In [28]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

In [34]:
def evaluateRandomly(encoder,pairs,input_lang,output_lang ,decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('Input:', pair[0])
        print('Target: ', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('Output:', output_sentence)
        print('')

In [ ]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = Encoder(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoder(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, 100, plot_every=5)

In [ ]:
encoder.eval()
decoder.eval()
input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
evaluateRandomly(encoder,pairs ,input_lang,output_lang,decoder, n=10)